In [23]:
import os
os.path.exists('/home/mz/code/MaCoZu/data')

True

The data set golub consists of the expression levels of 3051 genes for 38 tumor mRNA samples. Each tumor mRNA sample comes from one patient (i.e. 38 patients total), and 27 of these tumor samples correspond to acute lymphoblastic leukemia (ALL) and the remaining 11 to acute myeloid leukemia (AML). 

In [2]:
import zipfile
import numpy as np
import numpy.lib.recfunctions
import matplotlib.pyplot as plt

# Load data
with zipfile.ZipFile("/home/mz/code/MaCoZu/data/release_statsreview_release1.zip") as zip_file:
    golub_data, golub_classnames = ( np.genfromtxt(zip_file.open('data_and_materials/golub_data/{}'.format(fname)), delimiter=',', names=True, converters={0: lambda s: int(s.strip(b'"'))}) for fname in ['golub.csv', 'golub_cl.csv'] )

    ALL_names, AML_names = ( [ 'V{}'.format(int(v)) for v in golub_classnames[golub_classnames['x'] == c][golub_classnames.dtype.names[0]] ] for c in [0, 1] )
    ALL_genes, AML_genes, ALLAML_genes = ( numpy.lib.recfunctions.repack_fields(golub_data[names]).view((golub_data[names].dtype[0], len(names))) for names in [ALL_names, AML_names, ALL_names + AML_names] )
    ALL_N, AML_N, ALLAML_N = ( a.shape[1] for a in [ALL_genes, AML_genes, ALLAML_genes] )
    N_genes = ALLAML_genes.shape[0]


Missing colon in file PosixPath('/home/mz/.config/matplotlib/stylelib/scientific.mplstyle'), line 15 ("                               'fc4f30', ")
Missing colon in file PosixPath('/home/mz/.config/matplotlib/stylelib/scientific.mplstyle'), line 16 ("                               'e5ae38',")
Missing colon in file PosixPath('/home/mz/.config/matplotlib/stylelib/scientific.mplstyle'), line 17 ("                               '6d904f',")
Missing colon in file PosixPath('/home/mz/.config/matplotlib/stylelib/scientific.mplstyle'), line 18 ("                               '8b8b8b',")
Missing colon in file PosixPath('/home/mz/.config/matplotlib/stylelib/scientific.mplstyle'), line 19 ("                               '810f7c'])")
Bad value in file PosixPath('/home/mz/.config/matplotlib/stylelib/scientific.mplstyle'), line 14 ("axes.prop_cycle: cycler(color=['008fd5', "): Key axes.prop_cycle: 'cycler(color=['008fd5',' is not a valid cycler construction: unexpected EOF while parsing (<unknown>, 

 You will need to discover how many genes can be used to differentiate the tumor types, <br>
 meaning that their expression level differs between the two tumor types <br>
 (Acute Lymphoblastic Leukemia (ALL) and Acute Myeloid Leukemia (AML). ) using <br>

- the uncorrected p-values,
- the Holm-Bonferroni correction, and (iii) the Benjamini-Hochberg correction? 

## Uncorrected p-values

We can use $\Delta \bar{X}_ i = \bar{X}_{\text {ALL},i} - \bar{X}_{\text {AML},i}$ as a metric for the difference in expression levels for gene . 

The variance of this metric is $\displaystyle  s_{\Delta \bar{X}_ i}^2\displaystyle = s_{\bar{X}_{\text {ALL}},i}^2 + s_{\bar{X}_{\text {AML}},i}^2$



test-statistic:

$\displaystyle  t_{\text {Welch},i}\displaystyle = \frac{\bar{X}_{\text {ALL},i} - \bar{X}_{\text {AML},i}}{\sqrt{ \frac{s_{\text {ALL},i}^2}{N_{\text {ALL}}} + \frac{s_{\text {AML},i}^2}{N_{\text {AML}}} }}$

In [6]:
import scipy.stats
# Calculate the Welch's t-test statistic
test_statistic = (ALL_genes.mean(axis=1) - AML_genes.mean(axis=1))/np.sqrt(ALL_genes.var(axis=1, ddof=1)/ALL_N + AML_genes.var(axis=1, ddof=1)/AML_N)


## degrees of freedom

Welch test statistic can be approximated by a t-distribution, but with a modified number of **degrees of freedom** $v_i$.

$\displaystyle  \nu _ i\displaystyle \approx \frac{ \left(\frac{s_{\text {ALL},i}^2}{N_{\text {ALL}}} + \frac{s_{\text {AML},i}^2}{N_{\text {AML}}}\right)^2 }{ \frac{1}{\nu _{\text {ALL}}} \left(\frac{s_{\text {ALL},i}^2}{N_{\text {ALL}}}\right)^2 + \frac{1}{\nu _{\text {AML}}} \left( \frac{s_{\text {AML},i}^2}{N_{\text {AML}}} \right)^2 }.$

where $\nu _{\text {ALL}} = N_{\text {ALL}}-1$ <br>
and $\nu _{\text {AML}} = N_{\text {AML}}-1$

In [7]:
# Find the number of degrees of freedom of the statistic according to the approximation formula
test_dof = (ALL_genes.var(axis=1, ddof=1)/ALL_N + AML_genes.var(axis=1, ddof=1)/AML_N)**2 / \
    ( (ALL_genes.var(axis=1, ddof=1)/ALL_N)**2/(ALL_N-1) + (AML_genes.var(axis=1, ddof=1)/AML_N)**2/(AML_N-1))

## 2-sided p-values
sf : survival funtion = 1-cdf gives *p*-value for $t<x$ probability that we'll find an x (avarage observation) that's greater than t (the observation we've made this time) <br>
thus it's a one-sided *p*-value.  For two-sided *p*-value $[x<-t \,$  &  $\, +t <x]$ we double the one_sided *p*-value.

In [8]:
# Find the 2-sided p-values
p_values = scipy.stats.t.sf(np.abs(test_statistic), test_dof)*2

In [9]:
# Count how many of these p-values are below the significance threshold
(p_values < 0.05).sum()

1078

## Holm Bonferroni correction

The **Bonferroni correction**.<br>
 In a series of $m$ tests, if the significance level of each test is set to $\alpha/m$, <br>
 or equivalently if the null hypothesis $H^i_0$ of each test $i$  is rejected when the corresponding $p$-value <br>
 is bounded by: $\displaystyle  \displaystyle p^{i}< \frac{\alpha }{m},$ then $FWER< \alpha$. 
 
 Equivalently, to control $FWER< \alpha$, we can “correct" the $p$-value $p_i$ of each test to $mp_i$ and reject $H^i_0$ when $mp_i<\alpha$.<br>
The Bonferroni correction apples to a series of tests that are not necessary independent of each other. <br>

**FWER**: (family-wise error rate) is the probability of making one or more false discoveries, or type I errors when performing multiple hypotheses tests. <br><br>



**Holm-Bonferroni Correction**<br>
The Holm-Bonferroni method makes adaptive adjustments to the rejection criterion of each test. The procedure is as follows.<br>
Suppose we are testing for $m$ hypotheses. 

Sort the $m$  $p$-values in increasing order  $p^{(1)}\leq p^{(2)}\leq \ldots \leq p^{(i)}\leq \ldots \leq p^{(m)} \quad$  Start with $i=1$, i.e. the minimum $p$-value. <br>
If $\quad {\color{orange}{p^{(i)}\leq \frac{\alpha }{m-(i-1)}} = \color{orange}\frac{\alpha }{m+1-i}} ,$ then reject $H_0^{(i)}$ proceed to the next smallest $p$-value, increment $i$ by $1$, and again use the rejection criterion (in orange) above. 

In [10]:
# Sort the p-values in ascending order
p_values_sorted = np.sort(p_values)
# These are the adjusted significance thresholds as an array.
# Each element is the threshold for the corresponding p-value in p_values_sorted
# Note that (np.arange(N_genes)+1)[::-1] gives an array with [N_genes, N_genes-1, N_genes-2, ..., 1]
# The [::-1] reverses the array.
holm_bonferroni_thresholds = 0.05/(np.arange(N_genes)+1)[::-1]
# First we compare the p-values to the associated thresholds. We then get an array
# where the p-values that exceed the threhold have a value of False.
holm_bonferroni_significant = p_values_sorted < holm_bonferroni_thresholds
# We want to find the first value of False in this array (first p-value that exceeds the threshold)
# so we invert it using logical_not.
holm_bonferroni_not_significant = np.logical_not(holm_bonferroni_significant)
# argwhere will return an array of indices for values of True in the supplied array.
# Taking the first element of this array gives the first value of True in holm_bonferroni_not_significant
# which is the same as the first value of False in holm_bonferroni_significant
holm_bonferroni_first_not_significant = np.argwhere(holm_bonferroni_not_significant)[0]
# We reject all hypothesis before the first p-value that exceeds the significance threshold.
# The number of these rejections is exactly equal to the index of the first value that
# exceeds the threshold.
num_holm_bonferroni_rejections = holm_bonferroni_first_not_significant
print(num_holm_bonferroni_rejections)

[103]


## Benjamini Hochberg


Sometimes, controlling (the probability of making one or more false discoveries) may be too strict for any discovery to be reported. Instead, we can then control the **expected proportion of false discoveries among all discoveries made**, the false discovery rate (FDR). 

The false discovery rate (FDR) is:

 $\displaystyle  \text {FDR} = \mathbb {E}\left[\frac{V}{N_1}\right].$

 Recall **$N_1$ is the total number of discoveries** made (the total number of null hypotheses rejected), <br>
 and **$V$ is the number of false discoveries** (the number of null hypotheses that were falsely rejected). <br> 
 Hence $V\leq N_1$ and $V/N_1$ is a ratio that is always between zero and one. <br> 
 If no null hypotheses were rejected, i.e. if $N_1 = 0$, we define the ratio $V/N_1$ to be zero to avoid a division by zero. <br><br>

**Benjamini-Hochberg Correction**<br>
The Benjamini-Hochberg method guarantees $FDR < \alpha$ for a series of independent tests. 
Procedure: 
- Sort the $p$-values in increasing order $p^{(1)}\leq p^{(2)}\leq \ldots \leq p^{(i)}\leq \ldots \leq p^{(m)}$
- Such that: ${\color{orange}{p^{(k)}\leq \frac{k}{m}\alpha }}$.
- reject all of $H_0^{(1)}, H_0^{(2)}\cdots , H_0^{(k)}$


Note that in this procedure, some null hypothesis $H_0^{(i)}$ can be rejected even if $p^{(i)}>\frac{i}{m} \alpha$ provided that $i<k$. <br>

The adjustment of the p-value to ${\color{orange}{\frac{m}{i}p^{(i)}}}$ before comparing with $\alpha$ is called the **Benjamini-Hochberg correction**. 

In [15]:
benjamini_hochberg_thresholds

array([1.63880695e-05, 3.27761390e-05, 4.91642085e-05, ...,
       4.99672239e-02, 4.99836119e-02, 5.00000000e-02])

In [12]:
# These are the adjusted significance thresholds as an array.
benjamini_hochberg_thresholds = 0.05*(np.arange(N_genes)+1)/N_genes
# First we compare the p-values to the associated thresholds.
benjamini_hochberg_significant = p_values_sorted < benjamini_hochberg_thresholds
# We are intested in the last p-value which is significant.Remeber that argwhere returns an array of indicies for the True values, 
# so we take the last element in order to get the index of the last p-value which is significant.
benjamini_hochberg_last_significant = np.argwhere(p_values_sorted < benjamini_hochberg_thresholds)[-1]
# We reject all hypotheses before the last significant p-value, AND we reject the hypothesis for the last significant p-value as well. 
# So the number of rejected hypotheses is equal to the index of the last significant p-value PLUS one.
num_benjamini_hochberg_rejections = benjamini_hochberg_last_significant + 1
print(num_benjamini_hochberg_rejections)

[695]
